In [5]:
%matplotlib inline

import operator
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import jsonpickle
import json
import random
import tweepy
import spacy
import time
from datetime import datetime
from spacy.symbols import ORTH, LEMMA, POS

In [6]:
class Tweets:
    
    
    def __init__(self,term="",corpus_size=100):
        self.tweets={}
        if term !="":
            self.searchTwitter(term,corpus_size)
                
    def searchTwitter(self,term,corpus_size):
        searchTime=datetime.now()
        # get 1/4 of the corpus or 20 tweets, which ever is larger.
        search_size=20
        if corpus_size/4 > search_size:
            search_size = corpus_size/4
        while (self.countTweets() < corpus_size):
            new_tweets = api.search(term,lang="en",count=search_size)
            print("got "+str(len(new_tweets))+" candidate tweets")
            for nt_json in new_tweets:
                nt = nt_json._json
                if self.getTweet(nt['id_str']) is None and self.countTweets() < corpus_size:
                    self.addTweet(nt,searchTime,term)
            print("....."+str(len(self.tweets))+" tweets total")
            time.sleep(5)
                
    def addTweet(self,tweet,searchTime,term="",count=0):
        id = tweet['id_str']
        if id not in self.tweets.keys():
            self.tweets[id]={}
            self.tweets[id]['tweet']=tweet
            self.tweets[id]['count']=0
            self.tweets[id]['searchTime']=searchTime
            self.tweets[id]['searchTerm']=term
        self.tweets[id]['count'] = self.tweets[id]['count'] +1
        
    def getTweet(self,id):
        if id in self.tweets:
            return self.tweets[id]['tweet']
        else:
            return None
    
    def getTweetCount(self,id):
        return self.tweets[id]['count']
    
    def countTweets(self):
        return len(self.tweets)
    
    # return a sorted list of tupes of the form (id,count), with the occurrence counts sorted in decreasing order
    def mostFrequent(self):
        ps = []
        for t,entry in self.tweets.items():
            count = entry['count']
            ps.append((t,count))  
        ps.sort(key=lambda x: x[1],reverse=True)
        return ps
    
    # reeturns tweet IDs as a set
    def getIds(self):
        return set(self.tweets.keys())
    
    # save the tweets to a file
    def saveTweets(self,filename):
        json_data =jsonpickle.encode(self.tweets)
        with open(filename,'w') as f:
            json.dump(json_data,f)
    
    # read the tweets from a file 
    def readTweets(self,filename):
        with open(filename,'r') as f:
            json_data = json.load(f)
            incontents = jsonpickle.decode(json_data)   
            self.tweets=incontents
        
    def getSearchTerm(self,id):
        return self.tweets[id]['searchTerm']
    
    def getSearchTime(self,id):
        return self.tweets[id]['searchTime']
    
    def getText(self,id):
        tweet = self.getTweet(id)
        text=tweet['full_text']
        if 'retweeted_status'in tweet:
            original = tweet['retweeted_status']
            text=original['full_text']
        return text
                
    def addCode(self,id,code):
        tweet=self.getTweet(id)
        if 'codes' not in tweet:
            tweet['codes']=set()
        tweet['codes'].add(code)
        
   
    def addCodes(self,id,codes):
        for code in codes:
            self.addCode(id,code)
        
 
    def getCodes(self,id):
        tweet=self.getTweet(id)
        return tweet['codes']
    
    # NEW -ROUTINE TO GET PROFILE
    def getCodeProfile(self):
        summary={}
        for id in self.tweets.keys():
            tweet=self.getTweet(id)
            if 'codes' in tweet:
                for code in tweet['codes']:
                    if code not in summary:
                            summary[code] =0
                    summary[code]=summary[code]+1
        sortedsummary = sorted(summary.items(),key=operator.itemgetter(0),reverse=True)
        return sortedsummary

In [7]:
consumer_key='D2L4YZ2YrO1PMix7uKUK63b8H'
consumer_secret='losRw9T8zb6VT3TEJ9JHmmhAmn1GXKVj30dkiMv9vjhXuiWek9'
access_token='15283934-iggs1hiZAPI2o5sfHWMfjumTF7SvytHPjpPRGf3I6'
access_secret='bOvqssxS97PGPwXHQZxk83KtAcDyLhRLgdQaokCdVvwFi'

In [8]:
from tweepy import OAuthHandler

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

In [9]:
t1 = api.search("vaping",lang="en",count=10)

In [10]:
t1 = [t._json for t in t1]

In [11]:
t1ids = [t['id_str'] for t in t1]

In [12]:
t1ids

['1015041541881114631',
 '1015041444078342151',
 '1015041436444741632',
 '1015041308308602880',
 '1015041211286028289',
 '1015041137969491968',
 '1015041029538549760',
 '1015040966875443200',
 '1015040959669731334',
 '1015040958612819969']

In [13]:
t2 = api.search("vaping",lang="en",count=10)

In [14]:
t2 = [t._json for t in t2]

In [15]:
t2ids = [t['id_str'] for t in t2]

In [16]:
t2ids

['1015041541881114631',
 '1015041444078342151',
 '1015041436444741632',
 '1015041308308602880',
 '1015041211286028289',
 '1015041137969491968',
 '1015041029538549760',
 '1015040966875443200',
 '1015040959669731334',
 '1015040958612819969']

In [17]:
set(t1ids).intersection(t2ids)

{'1015040958612819969',
 '1015040959669731334',
 '1015040966875443200',
 '1015041029538549760',
 '1015041137969491968',
 '1015041211286028289',
 '1015041308308602880',
 '1015041436444741632',
 '1015041444078342151',
 '1015041541881114631'}

In [18]:
t3 = api.search("vaping",lang="en",count=50)

In [19]:
t4 = api.search("vaping",lang="en",count=50)

In [20]:
t3 = [t._json for t in t3]

In [21]:
t3ids = [t['id_str'] for t in t3]

In [22]:
t4 = [t._json for t in t4]

In [23]:
t4ids = [t['id_str'] for t in t4]

In [24]:
len(set(t3ids).intersection(t4ids))

49

In [25]:
t5 = api.search("vaping",lang="en",count=50)
t5ids= [t._json['id_str'] for t in t5]

In [26]:
len(set(t3ids).intersection(t5ids))

49

In [27]:
t6= api.search("vaping",lang="en",count=50)
t6ids= [t._json['id_str'] for t in t6]

In [28]:
len(set(t3ids).intersection(t6ids))

49

In [38]:
lists =[]
for i in range(10):
    res = api.search("vaping",lang="en",count=50)
    resids=[t._json['id_str'] for t in res]
    lists.append(resids)
    time.sleep(120)

for i in range(1,10):
    prior = lists[i-1]
    cur=lists[i]
    newids = set(prior).intersection(cur)
    unique = len(cur)-len(newids)
    print(unique)

4
7
2
8
4
7
0
3
2
